In [3]:
import pandas as pd

model_names = ["gpt-3.5-turbo", "gpt-4-turbo", "gpt-4o"]

for model_name in model_names:
    df=pd.read_csv(f"../../../results/{model_name}_jailbreak_responses_labeled_GeminiEval.csv", header=0)
    zero_ratio = (df['eval_label'] == 0).mean()
    print(zero_ratio)
    

0.6462633451957296
0.6704626334519573
0.8362989323843416
